# <B> Evaluation </B>
* Container: codna_pytorch_py39

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 설정

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
prefix = pm.get_params(key="PREFIX")

## 2.Processing-job for evaluation

In [4]:
import os
import sagemaker
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor

* **Set Up SageMaker Experiment**
    - Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

* params for processing job

In [5]:
from time import strftime
from smexperiments.trial import Trial
from smexperiments.experiment import Experiment

In [6]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [7]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

In [25]:
local_mode = False

if local_mode: 
    instance_type = 'local_gpu'
    
    import os
    from sagemaker.local import LocalSession
    from nemo.utils.notebook_utils import download_an4
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
else:
    instance_type = "ml.g4dn.8xlarge"
    sagemaker_session = sagemaker.Session()
    wav_s3_uri = os.path.join(preprocessing_data_path, "an4", "wav")

sagemaker_role_arn = pm.get_params(key=prefix + '-SAGEMAKER-ROLE-ARN')
preprocessing_data_path = pm.get_params(key=prefix + '-PREP-DATA-PATH')
bucket_name = pm.get_params(key=prefix + '-BUCKET')
eval_image_uri = pm.get_params(key=''.join([prefix, '-IMAGE-URI']))
model_artifact_s3_uri = pm.get_params(key=prefix + '-MODEL-PATH').rsplit("/", 1)[0]
model_artifact_s3_uri = pm.get_params(key=prefix + '-MODEL-PATH')
test_manifest_s3_path = os.path.join(preprocessing_data_path, "an4", "test_manifest.json")
wav_s3_uri = os.path.join(preprocessing_data_path, "an4", "wav")
eval_output_s3_uri = os.path.join("s3://", bucket_name, "evaluation", "output", "eval_output")

code_location= os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "evaluation",
    "backup_codes"
)

git_config = {
    'repo': f'https://{pm.get_params(key="-".join([prefix, "CODE_REPO"]))}',
    'branch': 'main',
    'username': pm.get_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), enc=True),
    'password': pm.get_params(key="-".join([prefix, "CODECOMMIT-PWD"]), enc=True)
}  

print (f"instance-type: {instance_type}")
print (f"image-uri: {eval_image_uri}")
print (f"role: {sagemaker_role_arn}")
print (f"bucket: {bucket_name}")
print (f"preprocessing-data-path: {preprocessing_data_path}")
print (f"model_artifact_s3_uri: {model_artifact_s3_uri}")
print (f"test_manifest_s3_path: {test_manifest_s3_path}")
print (f"wav_s3_uri: {wav_s3_uri}")
print (f"eval_output_s3_uri: {eval_output_s3_uri}")
print (f"sagemaker_session: {sagemaker_session}")
print (f"git_config: {git_config}")

instance-type: ml.g4dn.8xlarge
image-uri: 419974056037.dkr.ecr.us-east-1.amazonaws.com/nemo-test-training
role: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
bucket: sm-nemo-ramp
preprocessing-data-path: s3://sm-nemo-ramp/nemo-asr/preprocessing/data
model_artifact_s3_uri: s3://sm-nemo-ramp/nemo-asr/training/model-output/nemo-asr-nemo-experiments-0322-10521679482352/output/model.tar.gz
test_manifest_s3_path: s3://sm-nemo-ramp/nemo-asr/preprocessing/data/an4/test_manifest.json
wav_s3_uri: s3://sm-nemo-ramp/nemo-asr/preprocessing/data/an4/wav
eval_output_s3_uri: s3://sm-nemo-ramp/evaluation/output/eval_output
sagemaker_session: <sagemaker.session.Session object at 0x7fac0afcf910>
git_config: {'repo': 'https://git-codecommit.us-east-1.amazonaws.com/v1/repos/nemo-code', 'branch': 'main', 'username': 'dongjin-at-419974056037', 'password': 'wtLv/fP4ESjBDnyW5xgqFPGR0dMTIyK5/8gK6IS1Zsg='}


* Define processing job

In [26]:
eval_processor = FrameworkProcessor(
    estimator_cls=PyTorch,
    framework_version="1.13.1",
    role=sagemaker_role_arn, 
    image_uri=eval_image_uri,
    instance_count=1,
    instance_type=instance_type,
    code_location=code_location,
    env={
        'MANIFEST_PATH': '/opt/ml/input/data/testing/an4/wav', 
        'WAV_PATH' : '/opt/ml/processing/input/wav'
        }
    )

In [27]:
experiment_name = pm.get_params(key=prefix + "-EXPERI-NAME")
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

eval_processor.run(
    code="evaluate.py",
    source_dir="./code",
    git_config=git_config,
    inputs=[
        ProcessingInput(
            source=model_artifact_s3_uri,
            input_name="model_artifact",
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=test_manifest_s3_path,
            input_name="test_manifest_file",
            destination="/opt/ml/processing/input/manifest"
        ),
        ProcessingInput(
            source=wav_s3_uri,
            input_name="wav_dataset",
            destination="/opt/ml/processing/input/wav"
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="eval-output-data",
            source="/opt/ml/processing/evaluation",
            destination=eval_output_s3_uri
            
        ),
    ],
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=True
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
Cloning into '/tmp/tmp860zcx6q'...
remote: Counting objects: 20, done.        
Already on 'main'
INFO:sagemaker.processing:Uploaded /tmp/tmp860zcx6q/./code to s3://sm-nemo-ramp/nemo-asr/evaluation/backup_codes/nemo-asr-nemo-experiments-0322-12071679486840/source/sourcedir.tar.gz


Your branch is up to date with 'origin/main'.


INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-ramp/nemo-asr/evaluation/backup_codes/nemo-asr-nemo-experiments-0322-12071679486840/source/runproc.sh
INFO:sagemaker:Creating processing-job with name nemo-asr-nemo-experiments-0322-12071679486840


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateProcessingJob operation: The account-level service limit 'ml.g4dn.8xlarge for processing job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

In [ ]:
eval_processor

In [13]:
pm.put_params(key="-".join([prefix, "EVAL-OUTPUT-PATH"]), value=eval_output_s3_uri+"evaluation.json", overwrite=True)

'Store suceess'

In [14]:
pm.get_params(key="-".join([prefix, "EVAL-OUTPUT-PATH"]))

's3://sm-nemo-ramp/evaluation/output/eval_outputevaluation.json'

In [15]:
pm.get_params(key="-".join([prefix, "BUCKET"]))

'sm-nemo-ramp'